### MAKE AI THAT DETECTS ROCK PAPER SCISSORS FROM IMAGE
#### AND BUILD A MESSED UP CNN MODEL LUCKILY WORKS WELL WITH THE DATA
#### TRAINED DATA FROM https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip
#### And idk how to build ai so this is just hit or miss

In [ ]:
# Install Tensorflow
%pip install tensorflow
# Install Pillow
%pip install pillow
# Install Scipy
%pip install scipy
# Install Matplotlib
%pip install matplotlib

In [ ]:
# Make directory with os
import os
os.mkdir("./resources")

In [ ]:
# Download the datasheet
!wget --no-check-certificate \
    https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip \
    -O ./resources/rockpaperscissors.zip

In [ ]:
# Unzip the datasheet
import zipfile
uzip = zipfile.ZipFile("./resources/rockpaperscissors.zip", 'r')
uzip.extractall("./resources/")
uzip.close()

In [ ]:
# Clean up the confusing file / folder
import shutil, os

# Delete __macosx folder with the data
if os.path.exists('./resources/__MACOSX'):
    shutil.rmtree('./resources/__MACOSX')

# Rename folder to datasheet
if os.path.exists('./resources/rockpaperscissors'):
    os.rename('./resources/rockpaperscissors', "./resources/datasheet")

# Remove rps-cv-images
if os.path.exists('./resources/datasheet/rps-cv-images'):
    shutil.rmtree('./resources/datasheet/rps-cv-images')

In [ ]:
# Make Structure the Datasheet
import os
import random
FOLDER_LIST = ["train","validate","logs"]
for folder in FOLDER_LIST:
    if not os.path.exists('./resources/datasheet/' + folder):
        os.mkdir('./resources/datasheet/' + folder)

# A Bit Calculations
# REQUIRED:
#   2188 Number of images
#   1314 Number of Images for train
#   874 Number of Images for validate
#   3 Number of Classes
TRAIN_GET_PHOTO_PER_FOLDER = int(1314 / 3)
VALIDATE_GET_PHOTO_PER_FOLDER = int(874 / 3) # because it is not even
SHOULDBETOTAL = TRAIN_GET_PHOTO_PER_FOLDER * 3 + VALIDATE_GET_PHOTO_PER_FOLDER * 3 + 1

print("TRAIN_GET_PHOTO_PER_FOLDER:", TRAIN_GET_PHOTO_PER_FOLDER)
print("VALIDATE_GET_PHOTO_PER_FOLDER:", VALIDATE_GET_PHOTO_PER_FOLDER, "+1")
print("Total :", SHOULDBETOTAL)

ORIGIN_LIST = ["rock","paper","scissors"]
for folder in ORIGIN_LIST:
    if not os.path.exists('./resources/datasheet/validate/' + folder):
        os.mkdir('./resources/datasheet/validate/' + folder)
    else:
        print("Folder already exists:", folder)
        break
    for i in range(1,VALIDATE_GET_PHOTO_PER_FOLDER + 1):
        files = os.listdir('./resources/datasheet/' + folder)
        filea = random.choice(files)
        file = "./resources/datasheet/"+folder+"/"+filea
        shutil.move(file, "./resources/datasheet/validate/"+folder+"/"+filea)

# Get 1 file from 1 class for reach the required number of images
CHOSSEDFOLDER = ORIGIN_LIST[random.choice(range(3))]
files = os.listdir('./resources/datasheet/' + CHOSSEDFOLDER)
filea = random.choice(files)
file = "./resources/datasheet/"+CHOSSEDFOLDER+"/"+filea
shutil.move(file, "./resources/datasheet/validate/"+CHOSSEDFOLDER+"/"+filea)

for folder in ORIGIN_LIST:
    if not os.path.exists('./resources/datasheet/train/' + folder):
        os.mkdir('./resources/datasheet/train/' + folder)
    else:
        print("Folder already exists:", folder)
        break
    MANYDATALEFT = len(os.listdir('./resources/datasheet/' + folder))
    for i in range(1,MANYDATALEFT + 1):
        files = os.listdir('./resources/datasheet/' + folder)
        filea = random.choice(files)
        file = "./resources/datasheet/"+folder+"/"+filea
        shutil.move(file, "./resources/datasheet/train/"+folder+"/"+filea)

#EXPECTED:
#   2188 Number of images
#   438 Number of Images for train / folder
#   291 Number of Images for validate / folder and 1 file in random class
#   3 Number of Classes
counter = 0
for folder in ORIGIN_LIST:
    counter += len(os.listdir('./resources/datasheet/train/' + folder))
    counter += len(os.listdir('./resources/datasheet/validate/' + folder))
print("TOTAL ???", counter)

if counter != counter:
    print("ERROR: UH OH MISSING SOME DATA")

# CleanUp
for folder in ORIGIN_LIST:
    shutil.rmtree('./resources/datasheet/' + folder)

In [ ]:
# Checking tensorflow
import tensorflow as tf
print(tf.__version__)

In [ ]:
import os
# Vars
BASEDIR = "./resources/datasheet"
TRAIN_DIR = os.path.join(BASEDIR, 'train')
VALIDATE_DIR = os.path.join(BASEDIR, 'validate')

In [ ]:
# Listing
print(os.listdir(TRAIN_DIR))
print(os.listdir(VALIDATE_DIR))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
    )

validate_data_gen = ImageDataGenerator(
    rescale=1./255
    )

In [ ]:
train_generator = train_data_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=(300, 200),
    batch_size=20,
    class_mode='categorical',
    classes=['paper', 'rock', 'scissors']
)

validate_generator = validate_data_gen.flow_from_directory(
    VALIDATE_DIR,
    target_size=(300, 200),
    batch_size=20,
    class_mode='categorical',
    classes=['paper', 'rock', 'scissors']
)

In [ ]:
import matplotlib.pyplot as plt
for traingen in train_generator:
    images = traingen[0]
    labels = traingen[1]
    for i in range(len(images)):
        plt.subplot(1, 3, i+1)
        plt.imshow(images[i])
        plt.title(labels[i])
        plt.show()
        break

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 200, 3)),
    tf.keras.layers.MaxPooling2D(3, 3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam", 
    metrics=["accuracy"]
    )

In [ ]:
model.summary()

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=20,
    epochs=200,
    validation_data=validate_generator,
    validation_steps=3,
    verbose=1
)

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

FOLDER_LIST = ["rock","paper","scissors"]
for i in range(5):
    folder = FOLDER_LIST[random.choice(range(3))]
    print(folder)
    files = os.listdir('./resources/datasheet/validate/' + folder)
    filea = random.choice(files)
    file = "./resources/datasheet/"+folder+"/"+filea
    print(file)
    img = image.load_img(file, target_size=(300, 200))
    y = image.img_to_array(img)
    y = np.expand_dims(y, axis=0)
    y = y / 255

    predict = model.predict(y)

    label = train_generator.class_indices
    label = dict((v,k) for k,v in label.items())

    plt.imshow(img)
    plt.title(label[np.argmax(predict)])
    plt.show()

In [ ]:
#get model accuracy
model.evaluate(validate_generator)